In [ ]:
import os
import sys
import time

import dotenv
from IPython.display import HTML
import numpy as np
import pandas as pd
import requests

dotenv.load_dotenv("../.env")

EVALAP_API_URL = "http://localhost:8000/v1"
EVALAP_API_KEY = os.getenv("EVALAP_API_KEY")
ALBERT_API_URL = "https://albert.api.etalab.gouv.fr/v1"
ALBERT_API_KEY = os.getenv("ALBERT_API_KEY")
OPENAI_URL = "https://api.openai.com/v1"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
headers = {"Authorization": f"Bearer {EVALAP_API_KEY}"}

In [ ]:
import json
import pandas as pd
from pathlib import Path

# Get all JSON files
json_files = Path("_data/LegalBenchRAG/benchmarks").glob("*.json")

# Process files
rows = []
for json_file in json_files:
    data = json.load(open(json_file, 'r', encoding='utf-8'))

    for test in data['tests']:
        rows.append({
            'query': test['query'],
            'output_true': '\n\n'.join([s['answer'] for s in test['snippets']]),
            'snippets': test['snippets'],
            'dataset_name': json_file.stem
        })

# Create DataFrame
df = pd.DataFrame(rows)

# Quick overview
print(f"Total rows: {len(df)}")
print(f"Datasets: {df['dataset_name'].nunique()}")
df

In [ ]:
# Publish a dataset
dataset = {"name": "LegalBenchRAG", "readme": "see https://github.com/zeroentropy-ai/legalbenchrag"
           , "default_metric" : "judge_precision"
           , "df": df.to_json()}
response = requests.post(f'{EVALAP_API_URL}/dataset', json=dataset, headers=headers)
resp = response.json()
resp